# Forecasting Consensus Expectations: Nonfarm Payrolls (NFP) 

## Data preprocessing

**Imports**

In [255]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from glob import glob

from tqdm.auto import tqdm
from scipy import stats, special
from scipy.optimize import brentq
from collections import defaultdict
from itertools import product
from scipy.stats import t as student_t, norm, binomtest, jarque_bera

**Helper functions**

In [256]:
def load_hist_nfp(path: str) -> pd.DataFrame:
    """
    Loads and pivots Bloomberg's historical NFP data to a long dataframe format (one row per economist forecast).
    """
    raw = pd.read_excel(path, header=None, engine="openpyxl")
    
    #===============================CHECKS===================================#
    # check data sufficiency 
    assert raw.shape[0] >= 4, "Expected >= 4 rows."
    
    name_row, firm_row = raw.iloc[1], raw.iloc[2]
    
    # check that we have the correct ticker (NFP TCH)
    assert str(raw.iloc[0,0]).strip().upper().startswith("NFP"), \
        "Cell A1 should start with NFP. Wrong file loaded."
        
    # check for median and actual values 
    assert name_row.str.contains(r"\bActual\b", case=False, na=False).sum() == 1, \
        "Expected exactly one 'Actual' column in header row."
    assert name_row.str.contains(r"\bMedian\b", case=False, na=False).sum() == 1, \
        "Median column missing."
        
    # “Economist / Firm” must line up in the spreadsheet for proper data loading
    econ_mask = name_row.str.contains(r"\bEconomist\b", case=False, na=False)
    assert econ_mask.sum() == 1, "Missing (or duplicate) 'Economist' header."
    econ_col  = econ_mask.idxmax()
    assert str(firm_row.iloc[econ_col]).strip().lower() == "firm", \
        f"'Firm' label expected right under 'Economist' in row 3, col {econ_col+1}."

    # Every forecast column must be preceded by an “As of” column
    skip_tokens = {"median", "average", "survey", "high",
                   "low", "previous", "prior", "actual", "economist"}

    for col in range(2, raw.shape[1]):
        hdr = name_row[col]
        if (not isinstance(hdr, str) or
            any(tok in hdr.lower() for tok in skip_tokens)):
            continue  # not a forecaster column

        assert pd.notna(firm_row[col-1]) and \
               "as of" in str(firm_row[col-1]).lower(), \
               f"Column {col} ('{hdr}') is missing its preceding 'As of' column."
            
    #========================================================================================

    # find summary columns 
    col_actual = name_row.str.contains("Actual", case=False, na=False).idxmax()
    col_median = name_row.str.contains("Median", case=False, na=False).idxmax()

    dates  = pd.to_datetime(raw.iloc[3:, 0])
    period = pd.to_datetime(raw.iloc[3:, 1])
    actual = pd.to_numeric(raw.iloc[3:, col_actual], errors="coerce")
    median = pd.to_numeric(raw.iloc[3:, col_median], errors="coerce")

    base = pd.DataFrame(
        {"release_date": dates,
         "period":       period,
         "actual":       actual,
         "median_forecast": median}
    )

    skip_tokens = {"median", "average", "survey", "high",
                   "low", "previous", "prior", "actual"}
    frames = []
    ASOF_FMT = "%m/%d/%Y"

    for col in range(2, raw.shape[1]):
        hdr = name_row[col]
        if (not isinstance(hdr, str)
            or any(tok in hdr.lower() for tok in skip_tokens)):
            continue

        tmp = base.copy()
        tmp["economist"] = hdr
        tmp["firm"]      = firm_row[col]
        tmp["forecast"]  = pd.to_numeric(raw.iloc[3:, col], errors="coerce")
        tmp["asof"]      = pd.to_datetime(raw.iloc[3:, col-1],
                                          format=ASOF_FMT, errors="coerce")
        frames.append(tmp)

    return (pd.concat(frames, ignore_index=True)
              .dropna(subset=["forecast"]))

In [257]:
def load_single_release(raw_path: str,
                        release_date: str,
                        period_start: str) -> pd.DataFrame:
    """
    Converts a single Bloomberg NFP release into the long table layout for merge.
    """
    raw = pd.read_excel(raw_path, sheet_name=0, header=None,
                        engine="openpyxl")
    
    # check for correct single release workbook 
    assert str(raw.iloc[0, 0]).strip().upper().startswith("NFP"), \
        "Cell A1 must start with 'NFP' – wrong file/ticker."
        
    
    # --- forecaster block ------------------------------------------------
    fcst_num = pd.to_numeric(raw[3], errors="coerce")
    mask = raw[0].isna() & fcst_num.notna()

    block = (raw.loc[mask, [1, 2, 3, 4, 5]]
               .rename(columns={1: "economist",
                                2: "firm",
                                3: "forecast",
                                4: "asof",
                                5: "rank"}))

    block["forecast"] = fcst_num[mask]
    block["asof"]     = pd.to_datetime(block["asof"], errors="coerce")

    # fallback: if economist name missing, use firm in parentheses
    block["economist"] = block.apply(
        lambda r: r["economist"] if pd.notna(r["economist"])
        else f"({r['firm']})", axis=1
    )

    # --- actual print ----------------------------------------------------
    actual_mask = raw.apply(
        lambda r: r.astype(str).str.contains("Actual", case=False).any(),
        axis=1
    )
    vals = (pd.to_numeric(raw.loc[actual_mask].stack(), errors="coerce")
          .dropna())
    actual_val = vals.iloc[0] if len(vals) else np.nan

    rel_date = pd.to_datetime(release_date)
    if rel_date > pd.Timestamp.today().normalize():
        actual_val = np.nan       # future month – unknown actual yet
        
    # --- median print ----------------------------------------------------
    median_row_idx = raw.apply(
        lambda r: r.astype(str).str.contains(r"\bMedian\b", case=False, na=False).any(),
        axis=1
    ).idxmax()          # idx of the first matching row (0 if none found)

    if median_row_idx == 0 and not raw.iloc[0].astype(str).str.contains(r"\bMedian\b", case=False).any():
        median_val = np.nan          # no median row at all
    else:
        # Take that row, clean each cell, coerce to numeric, keep first valid number
        def _clean(x):
            s = str(x).replace(",", "").replace("k", "").replace("K", "").strip()
            return pd.to_numeric(s, errors="coerce")

        median_series = raw.loc[median_row_idx].apply(_clean).dropna()
        median_val = median_series.iloc[0] if len(median_series) else np.nan
    # --------------------------------------------------------------------
    block["median_forecast"] = median_val

    # --- tidy output -----------------------------------------------------
    tidy = (block.assign(release_date=rel_date,
                         period=pd.to_datetime(period_start),
                         actual=actual_val)
                 .loc[:, ["release_date", "period", "economist",
                          "firm", "forecast", "median_forecast", "actual", "asof"]])

    return tidy


In [258]:
def build_nfp_long(hist_path: str,
                   singles: list[tuple[str, str, str]] | None = None
) -> pd.DataFrame:
    """
    Returns a long NFP panel where any release_date that also appears in `singles` is fully replaced by the newer file.
    """
    nfp_long = load_hist_nfp(hist_path)

    if singles:
        for fp, rdate, pstart in singles:
            if not os.path.exists(fp):
                continue

            single_df   = load_single_release(fp, rdate, pstart)
            rel_date_dt = pd.to_datetime(rdate).normalize()

            # Drop all historical rows for that release date
            nfp_long = nfp_long[nfp_long["release_date"] != rel_date_dt]

            # append the refreshed single‑release rows
            nfp_long = pd.concat([nfp_long, single_df], ignore_index=True)

    return nfp_long

**Build NFP long dataframe + run checks**

In [259]:
# Example single release tuple format, (file_path, release_date, observation_date)
# Observational date should be month end of previous month (e.g. last day of July for August NFP release)
# SINGLE_RELEASES: list[tuple[str, str, str]] = [
#     # ("../raw/nfp_aug25.xlsx", "2025-09-05", "2025-08-01"),
# ]

In [260]:
HIST_PATH = "../raw/nfp_historical.xlsx"

SINGLE_RELEASES = []        # specify single releases here if applicable, format as above
# SINGLE_RELEASES = [
#     ("../RAW/nfp_aug.xlsx", "2025-08-01", "2025-07-31")
# ]
nfp_long = build_nfp_long(HIST_PATH, singles=SINGLE_RELEASES)
nfp_long.head()

,release_date,period,actual,median_forecast,economist,firm,forecast,asof
367,1998-12-04,1998-11-30,267.0,160.0,Peter Dixon,NaN,160.0,1998-12-04
368,1999-01-08,1998-12-31,378.0,200.0,Peter Dixon,NaN,200.0,1999-01-08
369,1999-02-05,1999-01-31,245.0,140.0,Peter Dixon,NaN,140.0,1999-02-05
370,1999-03-05,1999-02-28,275.0,245.0,Peter Dixon,NaN,220.0,1999-03-05
371,1999-04-02,1999-03-31,46.0,150.0,Peter Dixon,NaN,160.0,1999-03-29


In [261]:
# sort and keep last forecast per economist
nfp_last = (nfp_long
            .sort_values(["release_date", "economist", "asof"])
            .groupby(["release_date", "economist"], as_index=False)
            .tail(1)
            .reset_index(drop=True))

# nfp surprise
nfp_last["surprise"] = nfp_last["actual"] - nfp_last["median_forecast"]
nfp_last["error"] = nfp_last["forecast"] - nfp_last["actual"]


nfp_last.head()

,release_date,period,actual,median_forecast,economist,firm,forecast,asof,surprise,error
0,1997-08-01,1997-07-31,316.0,70.0,Anthony Chan,JPMorgan Chase Bank,75.0,1997-08-01,246.0,-241.0
1,1997-08-01,1997-07-31,316.0,70.0,Bruce Steinberg,Merrill Lynch & Co Inc,55.0,1997-08-01,246.0,-261.0
2,1997-08-01,1997-07-31,316.0,70.0,Crandall/Jordan,Wrightson ICAP LLC,125.0,1997-08-01,246.0,-191.0
3,1997-08-01,1997-07-31,316.0,70.0,David H Resler,Nomura Securities Intl Inc,65.0,1997-08-01,246.0,-251.0
4,1997-08-01,1997-07-31,316.0,70.0,David J Greenlaw,Morgan Stanley & Co LLC,100.0,1997-08-01,246.0,-216.0


In [262]:
# sanity checks

# {release_date, economist} pairs should be unique
assert nfp_last.duplicated(["release_date", "economist"]).sum() == 0

# asof has to strictly precede or be equal to release date 
valid_asof = nfp_last["asof"].notna()
assert (nfp_last.loc[valid_asof, "asof"]
        <= nfp_last.loc[valid_asof, "release_date"]).all()

# should have no more than one unique realized print per date 
assert (nfp_last.groupby("release_date")["actual"].nunique() <= 1).all()

# assert periods are month end 
assert nfp_last["period"].dt.is_month_end.all(), \
       "Found a period that is not month‑end!"

In [263]:
nfp_last.head()

,release_date,period,actual,median_forecast,economist,firm,forecast,asof,surprise,error
0,1997-08-01,1997-07-31,316.0,70.0,Anthony Chan,JPMorgan Chase Bank,75.0,1997-08-01,246.0,-241.0
1,1997-08-01,1997-07-31,316.0,70.0,Bruce Steinberg,Merrill Lynch & Co Inc,55.0,1997-08-01,246.0,-261.0
2,1997-08-01,1997-07-31,316.0,70.0,Crandall/Jordan,Wrightson ICAP LLC,125.0,1997-08-01,246.0,-191.0
3,1997-08-01,1997-07-31,316.0,70.0,David H Resler,Nomura Securities Intl Inc,65.0,1997-08-01,246.0,-251.0
4,1997-08-01,1997-07-31,316.0,70.0,David J Greenlaw,Morgan Stanley & Co LLC,100.0,1997-08-01,246.0,-216.0


In [264]:
# Beginning of modern birth-death 
df_full = (nfp_last
      .query("release_date >= '2003-06-01'"))

In [269]:
df_firms_full = (
    df_full
      .groupby(['release_date', 'period', 'firm'], as_index=False)
      .agg(actual           = ('actual',           'first'),
           median_forecast  = ('median_forecast',  'first'),
           forecast         = ('forecast',         'median'),
           asof             = ('asof',             'min'))
      .sort_values(['release_date', 'firm'])
)

# ---------- recompute derived columns ----------
df_firms_full['surprise'] = df_firms_full['actual'] - df_firms_full['median_forecast']
df_firms_full['error']    = df_firms_full['forecast'] - df_firms_full['actual'] 

# ---------- 3. sanity check ----------
assert not df_firms_full.duplicated(['release_date', 'period', 'firm']).any()
df_firms_full.head()

# Ensure each (release_date, firm) combination is unique
assert not df_firms_full.duplicated(subset=["release_date", "firm"]).any(), (
    "Duplicate (release_date, firm) rows found!"
)

In [266]:
# covid‐adjusted subset (drops 2020‑2022)
df = (df_full
      .query("~release_date.between('2020-01-01', '2022-12-31')"))
df_firms = (df_firms_full
      .query("~release_date.between('2020-01-01', '2022-12-31')"))

In [267]:
# latest = nfp_last.query("release_date == '2025-08-01'")
# latest.head()

**Export**

In [268]:
OUT_DIR       = "../out"
DF_FILE       = "nfp_df.parquet"        # filtered (post‑COVID exclusions)
DF_FULL_FILE  = "nfp_df_full.parquet"   # full history
DF_FIRMS_FILE = "nfp_firms_df.parquet"
DF_FIRMS_FULL_FILE = "nfp_firms_df_full.parquet"

os.makedirs(OUT_DIR, exist_ok=True)

df.to_parquet(os.path.join(OUT_DIR, DF_FILE),
              engine="pyarrow", index=False)
df_full.to_parquet(os.path.join(OUT_DIR, DF_FULL_FILE),
                    engine="pyarrow", index=False)
df_firms.to_parquet(os.path.join(OUT_DIR, DF_FIRMS_FILE),
                    engine="pyarrow", index=False)
df_firms_full.to_parquet(os.path.join(OUT_DIR, DF_FIRMS_FULL_FILE),
                         engine="pyarrow", index=False)

print(f"Saved clean  ➜  {OUT_DIR}/{DF_FILE}")
print(f"Saved full   ➜  {OUT_DIR}/{DF_FULL_FILE}")

Saved clean  ➜  ../out/nfp_df.parquet
Saved full   ➜  ../out/nfp_df_full.parquet


In [270]:
df_firms

,release_date,period,firm,actual,median_forecast,forecast,asof,surprise,error
0,2003-06-06,2003-05-31,4CAST/Continuum Economics,-17.0,-30.0,-45.0,2003-05-30,13.0,-28.0
1,2003-06-06,2003-05-31,Aletti Gestielle Sgr Spa,-17.0,-30.0,-15.0,2003-06-04,13.0,2.0
2,2003-06-06,2003-05-31,BBVA Securities Inc,-17.0,-30.0,90.0,2003-05-30,13.0,107.0
3,2003-06-06,2003-05-31,BNP Paribas,-17.0,-30.0,-30.0,2003-05-30,13.0,-13.0
4,2003-06-06,2003-05-31,"Balyasny Asset Management, L.P.",-17.0,-30.0,-50.0,2003-05-30,13.0,-33.0
...,...,...,...,...,...,...,...,...,...
19245,2025-08-01,2025-07-31,Visa USA Inc,NaN,105.0,115.0,2025-07-25,NaN,NaN
19246,2025-08-01,2025-07-31,Wells Fargo Bank NA,NaN,105.0,110.0,2025-07-25,NaN,NaN
19247,2025-08-01,2025-07-31,Wilmington Trust Company,NaN,105.0,75.0,2025-07-28,NaN,NaN
19248,2025-08-01,2025-07-31,Wolfe Research LLC,NaN,105.0,80.0,2025-07-24,NaN,NaN
